In [ ]:
# Improve the original answer and generate a better answer 

In [1]:
import pandas as pd
import google.generativeai as genai
import dotenv
import time
import os
dotenv.load_dotenv()

True

In [2]:
raw_dataset = pd.read_csv('../dataset/cleaned_dataset.csv')
raw_dataset.head

<bound method NDFrame.head of                                               Question  \
0    Is a high school diploma required for an F-1 v...   
1             Is it important to memorize my SEVIS ID?   
2    Is proof of housing required at the port of en...   
3    What document does a school provide for an F-1...   
4    What if I plan to do research collaboration wi...   
..                                                 ...   
995              What happens if I fall out of status?   
996  How does an F-1 student apply for economic har...   
997  Do university provide any job search resources...   
998  How many months must I complete before I can a...   
999  Will poor attendance in my previous institutio...   

                                                Answer  
0      It depends on the academic program applied for.  
1    You should at least be familiar with it. Alway...  
2    Not necessarily but CBP officers can ask about...  
3                                           F

In [3]:
raw_dataset['Answer'].iloc[0]

'It depends on the academic program applied for.'

In [4]:
raw_dataset.shape

(1000, 2)

In [5]:
class DatasetAnsImprover:
    def __init__(self) -> None:
        api_key = os.getenv("GOOGLE_API_KEY")
        model_name = 'gemini-2.0-flash-exp'
        genai.configure(api_key=api_key)
        improve_answers_prompt = """
                            You are a leagal expert advisor. You will be given a question and an answer. You are required to improve the answer. Just reply with improved answer nothing else. Answer in a single paragraph in 3-4 sentences"""
        self.chat_model = genai.GenerativeModel(model_name, system_instruction=improve_answers_prompt)

    def improve_answer(self, question: str, answer:str) -> bool:
        prompt = f"""
        Question: {question}
        Answer: {answer}
        Improved Answer:        
        """
        response = self.chat_model.generate_content(prompt)
        return response.text.strip()


In [ ]:
# import pprint
# dataset_improver = DatasetAnsImprover()
# improved_answer = dataset_improver.improve_answer(raw_dataset['Question'].iloc[0], raw_dataset['Answer'].iloc[0])

# print('Original Answer:', raw_dataset['Answer'].iloc[0])
# print('Improved Answer:')
# pprint.pprint(improved_answer)

Original Answer: You must get approval from your Designated School Official (DSO) and submit a CPT request form along with an offer letter from your employer.
Improved Answer:
('To apply for CPT, start by obtaining a job offer that qualifies for CPT and '
 'then completing the CPT request form provided by your school. Next, you must '
 'secure approval from your Designated School Official (DSO), who will review '
 'your application and ensure it meets all requirements. Finally, submit the '
 'completed CPT request form, along with your offer letter and any other '
 'required documentation, to your DSO for processing and authorization.')


In [ ]:
# dataset_improver = DatasetAnsImprover()

# # loop over the dataset and improve the answers
# for index, row in raw_dataset.iterrows():
#     try:
#         improved_answer = dataset_improver.improve_answer(row['Question'], row['Answer'])
#         time.sleep(3)
#         if index % 10 == 0:
#             time.sleep(5)
#         raw_dataset.at[index, 'Answer'] = improved_answer
#     except Exception as e:
#         print(f"Error at index {index}: {e}")
#         time.sleep(10)
#         continue

# raw_dataset.to_csv('../dataset/cleaned_dataset_answer_improved.csv', index=False)

In [7]:
import pandas as pd
import time
import os

dataset_improver = DatasetAnsImprover()
output_file = '../dataset/cleaned_dataset_answer_improved.csv'
processed_rows = []
max_retries = 3  # Maximum number of retries for a failed question

for index, row in raw_dataset.iterrows():
    retries = 0
    while retries < max_retries:
        try:
            # improve_answer might fail as it makes a call to GEMINI API
            improved_answer = dataset_improver.improve_answer(row['Question'], row['Answer'])
            time.sleep(3)  # Sleep to avoid rate limiting
            if index % 15 == 0:
                time.sleep(10)  # Additional sleep for every 15th row
            
            # Create a new row with the improved answer
            new_row = row.to_dict()
            new_row['Answer'] = improved_answer
            processed_rows.append(new_row)
            
            # Save every 50 rows
            if len(processed_rows) == 50:
                print(f"Saving rows {index-49} to {index}")
                df_to_save = pd.DataFrame(processed_rows)
                # Check if the file exists to decide on writing the header
                header = not os.path.exists(output_file)
                df_to_save.to_csv(output_file, mode='a', header=header, index=False)
                processed_rows = []  # Reset the list
            
            break  # Exit the retry loop if successful
        
        except Exception as e:
            print(f"Error at index {index}: {e}")
            print("Failed question is : ", row['Question'])
            retries += 1
            if retries < max_retries:
                print(f"Retrying ({retries}/{max_retries}) after 10 seconds...")
                time.sleep(10)  # Wait before retrying
            else:
                print(f"Max retries reached for index {index}. Skipping this question.")
                break  # Exit the retry loop after max retries

# Save any remaining rows after the loop
if processed_rows:
    df_to_save = pd.DataFrame(processed_rows)
    header = not os.path.exists(output_file)
    df_to_save.to_csv(output_file, mode='a', header=header, index=False)

Error at index 11: 429 Resource has been exhausted (e.g. check quota).
Failed question is :  How do I know if my degree is considered STEM-eligible for the extension?
Retrying (1/3) after 10 seconds...
Error at index 32: 429 Resource has been exhausted (e.g. check quota).
Failed question is :  What if the program includes study abroad segments in another country?
Retrying (1/3) after 10 seconds...
Error at index 43: 429 Resource has been exhausted (e.g. check quota).
Failed question is :  Do I need a U.S. co-signer for my loan?
Retrying (1/3) after 10 seconds...
Saving rows 0 to 49
Error at index 65: 429 Resource has been exhausted (e.g. check quota).
Failed question is :  What if the school is delayed in issuing my I-20?
Retrying (1/3) after 10 seconds...
Error at index 87: 429 Resource has been exhausted (e.g. check quota).
Failed question is :  Can an F-1 visa holder attend any school?
Retrying (1/3) after 10 seconds...
Saving rows 50 to 99
Error at index 109: 429 Resource has been 

In [8]:
result = pd.read_csv(output_file)
result.shape

(1000, 2)